In [1]:
import nltk
nltk.download("gutenberg")


[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\91701\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
from nltk.corpus import gutenberg
import pandas as pd


In [3]:
# load the dataset
data = gutenberg.raw("shakespeare-hamlet.txt")
# sova to a file 
with open("hamlet.txt", "w") as f:
    f.write(data)


In [ ]:
# data preprocessing 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

ImportError: Traceback (most recent call last):
  File "C:\Users\91701\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [5]:
# load the dataset 
with open("hamlet.txt", "r") as f:
    text = f.read().lower()

In [6]:
# tokenization 
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(total_words)

4818


In [7]:
# creating input seq 
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

In [8]:
input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [9]:
import numpy as np 
max_seq = max([len(x) for x in input_sequences])
max_seq
input_sequences = np.array(pad_sequences(input_sequences,max_seq, padding="pre"))

In [10]:
max_seq

14

In [11]:
# create predictors and labels 
import tensorflow as tf
x, y = input_sequences[:,:-1], input_sequences[:,-1]

In [12]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
# define earlystopping 
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,GRU

In [16]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_seq-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

c:\python\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
##gru based model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_seq-1))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.build(input_shape=(None, max_seq-1))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 13, 150)        │       113,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,157,418 (4.42 MB)

 Trainable params: 1,157,418 (4.42 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[earlystopping],verbose = 1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.0308 - loss: 7.1747 - val_accuracy: 0.0323 - val_loss: 6.7928
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.0383 - loss: 6.5118 - val_accuracy: 0.0478 - val_loss: 6.7863
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0540 - loss: 6.2426 - val_accuracy: 0.0536 - val_loss: 6.7627
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0618 - loss: 6.0268 - val_accuracy: 0.0690 - val_loss: 6.7151
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.0819 - loss: 5.7268 - val_accuracy: 0.0701 - val_loss: 6.8225
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0908 - loss: 5.4576 - val_accuracy: 0.0715 - val_loss: 6.8570
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.1036 - loss: 5.2109 - val_accuracy: 0.0727 - val_loss: 6.9685


In [19]:
def predict_next_word(model, tokenizer, text,max_seq):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) > max_seq - 1:
        token_list = token_list[-(max_seq - 1):]
    token_list = pad_sequences([token_list], maxlen=max_seq - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predict_next_word = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predict_next_word:
            return word
    
    

In [20]:
input_text = "to be, or not to be, that is the"
print(f"Input text: {input_text}")
max_seq = model.input_shape[1] + 1
next_word = predict_next_word(model, tokenizer, input_text, max_seq)
print(f"Next Word : {next_word}")

Input text: to be, or not to be, that is the
Next Word : king


In [22]:
# save the model 
import pickle
model.save("next_word_lstm.h5")
# save the tokenizer 
with open ("tokenizer.pickle", "wb") as handle :
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
